In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

kevinwtan_leagueoflegendsmatchperformancepredictions_path = kagglehub.dataset_download('kevinwtan/leagueoflegendsmatchperformancepredictions')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torchmetrics
import torch
import torch.nn as nn
import torchvision
import torchmetrics
import numpy as np
from tqdm import tqdm
import sklearn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/kaggle/input/leagueoflegendsmatchperformancepredictions/team_match_dataset.jsonl", split="train")

len(dataset[0]["past_values"]), len(dataset[0]["future_values"])

In [ ]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
np.array(train_dataset[0]["past_values"]).shape

In [ ]:
from sklearn.preprocessing import StandardScaler

train_features = np.vstack([
    np.array(entry["past_values"] + entry["future_values"])
    for entry in train_dataset
])
scaler = StandardScaler()
scaler.fit(train_features)

print(train_features.shape)

def scale_example(example):
    example["past_values"] = scaler.transform(np.array(example["past_values"])).tolist()
    example["future_values"] = scaler.transform(np.array(example["future_values"])).tolist()
    return example

scaled_train_dataset = train_dataset.map(scale_example)
scaled_test_dataset = test_dataset.map(scale_example)

In [ ]:
import numpy as np
scaled_sample = scaled_train_dataset[0]
np.array(scaled_sample["past_values"]).mean(), np.array(scaled_sample["past_values"]).std()

In [ ]:
len(scaled_train_dataset), len(scaled_test_dataset)

In [ ]:
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel

feature_size = len(train_dataset[0]["past_values"][0])  # Size of a single feature vector

# Define the configuration for the model
configuration = TimeSeriesTransformerConfig(
    prediction_length=1,
    context_length=10,
    input_size=feature_size,
    lags_sequence=[1]
)

# Initialize the model with the configuration
model = TimeSeriesTransformerModel(configuration)

# Get the model's configuration
configuration = model.config

2025-05-13 20:42:19.158846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747168939.612342      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747168939.742266      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        entry = self.dataset[idx]
        return {
            "past_values": torch.tensor(entry["past_values"], dtype=torch.float32),
            "future_values": torch.tensor(entry["future_values"], dtype=torch.float32),
        }

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(TimeSeriesDataset(scaled_train_dataset), batch_size=32, shuffle=True)
test_loader = DataLoader(TimeSeriesDataset(scaled_test_dataset), batch_size=32)

In [ ]:
import torch.nn as nn

class LSTMForecaster(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMForecaster, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # x: [batch, seq_len, input_size]
        out = self.fc(lstm_out[:, -1, :])  # last time step
        return out

In [ ]:
# print(torch.cuda.is_available())       # Should be True
# print(torch.cuda.current_device())     # Should return 0 or a valid GPU id
# print(torch.cuda.get_device_name(0))   # Should print your GPU name

In [ ]:
input_size = len(scaled_train_dataset[0]["past_values"][0])
output_size = len(scaled_train_dataset[0]["future_values"][0])

input_size, output_size

In [ ]:
from torch.optim import AdamW

input_size = len(scaled_train_dataset[0]["past_values"][0])
output_size = len(scaled_train_dataset[0]["future_values"][0])
model = LSTMForecaster(input_size=input_size, hidden_size=64, num_layers=5, output_size=output_size)

lr = 1e-4
wd = 1e-3
epochs = 5

train_mses = []
test_mses = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

opt = AdamW(model.parameters(), lr=lr, weight_decay=wd)
loss_fn = torch.nn.MSELoss()

epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0

    pbar = tqdm(total=len(train_loader))
    for batch in train_loader:
        past = batch["past_values"].to(device)  # shape: [batch, seq_len, input_size]
        future = batch["future_values"].to(device)  # shape: [batch, output_size]

        opt.zero_grad()
        preds = model(past)
        # loss = loss_fn(preds, future)
        loss = loss_fn(preds, future.squeeze(1))
        loss.backward()
        opt.step()
        total_loss += loss.item()
        pbar.update(1)
    pbar.close()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_loss:.4f}")

    # Eval
    model.eval()
    with torch.no_grad():
        test_loss = 0
        pbar = tqdm(total=len(test_loader))
        for batch in test_loader:
            past = batch["past_values"].to(device)
            future = batch["future_values"].to(device)
            preds = model(past)
            test_loss += loss_fn(preds, future.squeeze(1)).item()
            pbar.update(1)
        pbar.close()
        print(f"Epoch {epoch+1}, Test Loss: {test_loss / len(test_loader):.4f}")

100%|██████████| 1167/1167 [00:58<00:00, 19.83it/s]


Epoch 1, Train Loss: 0.6504


100%|██████████| 292/292 [00:13<00:00, 21.25it/s]


Epoch 1, Test Loss: 0.5795


100%|██████████| 1167/1167 [00:58<00:00, 20.03it/s]


...

Epoch 9, Test Loss: 0.5088


100%|██████████| 1167/1167 [01:02<00:00, 18.79it/s]


Epoch 10, Train Loss: 0.5033


100%|██████████| 292/292 [00:14<00:00, 20.00it/s]

Epoch 10, Test Loss: 0.5082


In [ ]:
torch.save(model.state_dict(), "lstm_rnn_model_1.pth")

In [ ]:
min(min(scaled_train_dataset[0]["past_values"])), max(max(scaled_train_dataset[0]["past_values"])), min(min(scaled_train_dataset[0]["future_values"])), max(max(scaled_train_dataset[0]["future_values"]))

(-1.6223253693462107,
 5.619866001845678,
 -1.6223253693462107,
 5.619866001845678)

In [ ]:
print(next(model.parameters()).device)

cuda:0


In [ ]:
!nvidia-smi

Mon May 12 20:30:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             29W /   70W |     225MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from torch.optim import AdamW

input_size = len(scaled_train_dataset[0]["past_values"][0])
output_size = len(scaled_train_dataset[0]["future_values"][0])
model = LSTMForecaster(input_size=input_size, hidden_size=128, num_layers=10, output_size=output_size)

lr = 1e-4
wd = 1e-3

train_mses = []
test_mses = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

opt = AdamW(model.parameters(), lr=lr, weight_decay=wd)
loss_fn = torch.nn.MSELoss()

epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0

    pbar = tqdm(total=len(train_loader))
    for batch in train_loader:
        # past = batch["past_values"].to(device)  # shape: [batch, seq_len, input_size]
        past = torch.tensor(batch["past_values"], dtype=torch.float32).to(device)
        # future = batch["future_values"].to(device)  # shape: [batch, output_size]
        future = torch.tensor(batch["future_values"], dtype=torch.float32).to(device)

        opt.zero_grad()
        preds = model(past)
        # loss = loss_fn(preds, future)
        loss = loss_fn(preds, future.squeeze(1))
        loss.backward()
        opt.step()
        total_loss += loss.item()
        pbar.update(1)
    pbar.close()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_loss:.4f}")

    # Eval
    model.eval()
    with torch.no_grad():
        test_loss = 0
        pbar = tqdm(total=len(test_loader))
        for batch in test_loader:
            past = batch["past_values"].to(device)
            future = batch["future_values"].to(device)
            preds = model(past)
            test_loss += loss_fn(preds, future.squeeze(1)).item()
            pbar.update(1)
        pbar.close()
        print(f"Epoch {epoch+1}, Test Loss: {test_loss / len(test_loader):.4f}")

  0%|          | 0/1167 [00:00<?, ?it/s]/tmp/ipykernel_31/3101085749.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  past = torch.tensor(batch["past_values"], dtype=torch.float32).to(device)
/tmp/ipykernel_31/3101085749.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  future = torch.tensor(batch["future_values"], dtype=torch.float32).to(device)
100%|██████████| 1167/1167 [01:02<00:00, 18.73it/s]


Epoch 1, Train Loss: 0.6683


100%|██████████| 292/292 [00:14<00:00, 20.46it/s]


Epoch 1, Test Loss: 0.6464


100%|██████████| 1167/1167 [01:01<00:00, 18.88it/s]


...

Epoch 17, Train Loss: 0.5038


100%|██████████| 292/292 [00:14<00:00, 20.36it/s]


Epoch 17, Test Loss: 0.5092


 17%|█▋        | 196/1167 [00:10<00:49, 19.47it/s]

KeyboardInterrupt: 

In [ ]:
input_size, output_size

(237, 237)

In [ ]:
torch.save(model.state_dict(), "lstm_rnn_model_128_10.pth")

In [ ]:
# Recreate model with same architecture
model = LSTMForecaster(input_size=..., hidden_size=..., num_layers=..., output_size=...)

# Load saved weights
model.load_state_dict(torch.load("rnn_model.pth"))

# Move to GPU if needed
model.to(device)
model.eval()  # Set to evaluation mode if you're doing inference

In [ ]:
with torch.no_grad():
    example = torch.tensor(some_input_sequence, dtype=torch.float32).unsqueeze(0).to(device)
    prediction = model(example)

In [ ]:
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel

feature_size = len(train_dataset[0]["past_values"][0])  # Size of a single feature vector

# Define the configuration for the model
configuration = TimeSeriesTransformerConfig(
    prediction_length=1,
    context_length=10,
    input_size=feature_size,
    lags_sequence=[1]
)

# Initialize the model with the configuration
model = TimeSeriesTransformerModel(configuration)

# Get the model's configuration
configuration = model.config

In [ ]:
import torch
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

lr = 1e-4
wd = 1e-3
epochs = 5

opt = AdamW(model.parameters(), lr=lr, weight_decay=wd)
loss_fn = torch.nn.MSELoss()

train_mses = []
test_mses = []

best_test_mse = np.inf

for epoch in range(epochs):
    model.train()

    for batch in train_loader:
        opt.zero_grad()

        past_values = batch["past_values"].to(device)

        future_values = batch["future_values"].to(device)

        past_observed_mask = torch.ones_like(past_values).to(device)
        past_time_features = torch.zeros_like(past_values).to(device)
        future_time_features = torch.zeros_like(past_values).to(device)

        outputs = model(
            past_values=past_values,
            past_time_features=past_time_features,
            past_observed_mask=past_observed_mask,
            future_values=future_values,
            future_time_features=future_time_features
        )

        loss = loss_fn(outputs.predictions, future_values)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for batch in val_loader:
            past_values = batch["past_values"].to(device)
            future_values = batch["future_values"].to(device)

            outputs = model(past_values=past_values, future_values=future_values)
            loss = loss_fn(outputs.predictions, future_values)
            val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch+1} - Val Loss: {avg_val_loss:.4f}")

ValueError: lags cannot go further than history length, found lag 1 while history length is only 11

In [ ]:
torch.save(model.state_dict(), "timeseries_transformer.pth")